In [1]:
# === NumPy, SciPy e otimização ===
import numpy as np
from scipy.optimize import minimize
  
# === Visualização ===
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram, array_to_latex, plot_state_city
from rustworkx.visualization import mpl_draw as draw_graph
import time

# === Estruturas de grafos ===
import networkx as nx
import rustworkx as rx

# === Qiskit: circuitos, operadores e transpiler ===
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit.library import QAOAAnsatz, PauliEvolutionGate
from qiskit.synthesis.evolution import LieTrotter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

# === Qiskit Aer (simulação local) ===
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as EstimatorV2, Sampler as AerSampler

# === Qiskit Runtime (execução em hardware IBM) ===
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2, SamplerV2

# === Qiskit Primitives ===
# from qiskit.primitives import Sampler, BaseSamplerV2

# === Qiskit Optimization === 
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.minimum_eigensolvers import QAOA
from qiskit_optimization.optimizers import COBYLA, NELDER_MEAD

# ---- Importações necessárias (atualizadas para Qiskit 2.x) ----
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorEstimator as Estimator  # Correção: Use StatevectorEstimator
from qiskit.primitives import StatevectorSampler as Sampler  # Correção: Use StatevectorSampler
from qiskit.circuit.library import QAOAAnsatz  # Novo: QAOAAnsatz em vez de QAOA
from qiskit.quantum_info import SparsePauliOp  # Para operadores
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager  # transpiler para versões antigas/compatíveis
from qiskit import transpile
import numpy as np
from scipy.optimize import minimize  # Otimizador clássico externo 

from qiskit.providers.basic_provider import BasicSimulator

In [2]:
valor_itens = [44, 46, 90, 72, 91, 40, 75, 35, 8, 54, 78, 40, 77, 15, 61, 17, 75, 29, 75, 63]
peso_itens = [92, 4, 43, 83, 84, 68, 92, 82, 6, 44, 32, 18, 56, 83, 25, 96, 70, 48, 14, 58]
capacidade = 878
qtd_itens = len(valor_itens)
penalidade = max(valor_itens) * 10 

print(f"Capacidade: {capacidade}")
print(f"Peso: {peso_itens}, Quantidade: {qtd_itens}")
print(f"Valor: {valor_itens}, Quantidade: {qtd_itens}")
print(f"Penalidade: {penalidade}")
print(f"Total itens: {sum(valor_itens)}")
print(f"Total pesos: {sum(peso_itens)}")
if qtd_itens == len(peso_itens):
    print(f"OK: {qtd_itens}")
else:
    print("Lista itens diferente de lista pesos")       

Capacidade: 878
Peso: [92, 4, 43, 83, 84, 68, 92, 82, 6, 44, 32, 18, 56, 83, 25, 96, 70, 48, 14, 58], Quantidade: 20
Valor: [44, 46, 90, 72, 91, 40, 75, 35, 8, 54, 78, 40, 77, 15, 61, 17, 75, 29, 75, 63], Quantidade: 20
Penalidade: 910
Total itens: 1085
Total pesos: 1098
OK: 20


In [3]:
'''
Valor itens e pesos; 
'''
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade):
    '''
    Função QUBO
    '''
    n = len(valor_itens)
    qubo_matrix = np.zeros([n, n])
    
    # Primeiro, termos lineares e quadráticos no diag
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] + penalidade * (peso_itens[i]**2) - 2 * penalidade * peso_itens[i] * capacidade
    
    # Off-diag simétricos: +2 P w_i w_j pra i != j (mas só setamos i<j e copiamos)
    for i in range(n):
        for j in range(i + 1, n):
            q_ij = 2 * penalidade * peso_itens[i] * peso_itens[j]
            qubo_matrix[i, j] = q_ij
            qubo_matrix[j, i] = q_ij  # <- Isso corrige a simetria!
    
    # Sanity check
    if not np.allclose(qubo_matrix, qubo_matrix.T):
        print("AVISO: Matriz QUBO não simétrica!")
    
    # Teste rápido: custo pra x= todos 0 (deve ser 0)
    x_zero = np.zeros(n)
    custo_zero = np.dot(x_zero, np.dot(qubo_matrix, x_zero))
    print(f"Sanity: Custo pra x=0: {custo_zero} (deve ser ~0)")
    
    return qubo_matrix

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade)
print("Matriz QUBO (primeiras 5x5, corrigida):")
print(qubo_matrix[:5, :5])  # Mais legível que full 20x20

Sanity: Custo pra x=0: 0.0 (deve ser ~0)
Matriz QUBO (primeiras 5x5, corrigida):
[[-1.39310124e+08  6.69760000e+05  7.19992000e+06  1.38975200e+07
   1.40649600e+07]
 [ 6.69760000e+05 -6.37732600e+06  3.13040000e+05  6.04240000e+05
   6.11520000e+05]
 [ 7.19992000e+06  3.13040000e+05 -6.70297800e+07  6.49558000e+06
   6.57384000e+06]
 [ 1.38975200e+07  6.04240000e+05  6.49558000e+06 -1.26361762e+08
   1.26890400e+07]
 [ 1.40649600e+07  6.11520000e+05  6.57384000e+06  1.26890400e+07
  -1.27807771e+08]]


In [4]:
# (Assumindo qubo_matrix do anterior)
def hamiltoniano_de_custo(qubo_matrix):  # Nome corrigido
    n = len(qubo_matrix)
    h_custo = SparsePauliOp.from_list([], num_qubits=n)  # Inicia vazio (melhor que I com 0)
    
    # Lineares completos: h_k = -0.5 * sum_j Q_{k j} (full row sum)
    h = -0.5 * np.sum(qubo_matrix, axis=1)  # Vetor de h_k
    for k in range(n):
        if abs(h[k]) > 1e-10:  # Evita zeros numéricos
            pauli_str = ["I"] * n
            pauli_str[k] = "Z"
            h_custo += SparsePauliOp("".join(pauli_str), coeffs=[h[k]])
    
    # Quadráticos: só i < j, J_ij = Q_ij / 4
    for i in range(n):
        for j in range(i + 1, n):
            if abs(qubo_matrix[i, j]) > 1e-10:
                pauli_str = ["I"] * n
                pauli_str[i] = "Z"
                pauli_str[j] = "Z"
                coef = qubo_matrix[i, j] / 4
                h_custo += SparsePauliOp("".join(pauli_str), coeffs=[coef])
    
    # Sanity check simples: teste com x = [1, 0, ..., 0]
    x_test = np.zeros(n)
    x_test[0] = 1.0
    e_qubo = np.dot(x_test, np.dot(qubo_matrix, x_test))
    # Pra Ising: Z_0 = -1 (x=1), outros Z=1 (x=0), mas só linear h_0 * (-1)
    e_ising_approx = h[0] * (-1) + sum(h[k] for k in range(1,n))  # Aproximação sem full sim
    print(f"Sanity: E_QUBO (x=[1,0,...]): {e_qubo:.2f}")
    print(f"E_Ising approx: {e_ising_approx:.2f} (deve bater dentro de ~1e-10)")
    
    return h_custo

hamiltoniano_c = hamiltoniano_de_custo(qubo_matrix)
print("Hamiltoniano de custo (corrigido):")
print(hamiltoniano_c)  # Mostra os termos principais

Sanity: E_QUBO (x=[1,0,...]): -139310124.00
E_Ising approx: -155178161.50 (deve bater dentro de ~1e-10)
Hamiltoniano de custo (corrigido):
SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIIII', 'IIIIIIZIIIIIIIIIIIII', 'IIIIIIIZIIIIIIIIIIII', 'IIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIZ', 'ZZIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIII', 'ZIIIIIIIIIZIIIIIIIII', 'ZIIIIIIIIIIZIIIIIIII', 'ZIIIIIIIIIIIZIIIIIII', 'ZIIIIIIIIIIIIZIIIIII', 'ZIIIIIIIIIIIIIZIIIII', 'ZIIII

In [5]:
# (Assumindo valor_itens, peso_itens do topo)
def hamiltoniano_de_mixer(valor_itens, peso_itens, uniforme=False, normalize_sum=False):
    n = len(valor_itens)
    # Fix: Init vazio com num_qubits=n
    h_mix = SparsePauliOp.from_list([], num_qubits=n)
    
    if uniforme:
        coefs = [1.0] * n
        print("Usando mixer uniforme (todos coef=1.0)")
    else:
        razao = []
        for i in range(n):
            if peso_itens[i] == 0:
                razao.append(0.0)
                print(f"Aviso: peso[i={i}]=0, razao=0")
            else:
                razao.append(valor_itens[i] / peso_itens[i])
        
        # Fix sintaxe: max primeiro, check separado
        max_razao = max(razao) if razao else 0.0
        if max_razao == 0:
            max_razao = 1.0  # Fallback pra evitar div/0 total
            print("Aviso: Todas razao=0; usando uniforme implícito")
            coefs = [1.0] * n
        else:
            coefs = [r / max_razao for r in razao]
            if normalize_sum:
                sum_razao = sum(razao)
                if sum_razao > 0:
                    coefs = [r / sum_razao * n for r in razao]  # Soma exata = n
                    print("Normalizado por soma (total força = n)")
        
        # Debug: top 3 itens por razao
        top_idx = np.argsort(razao)[-3:][::-1]
        print(f"Top 3 itens por v/w: idx {top_idx}, razoes {[razao[i] for i in top_idx]}")
    
    for i in range(n):
        pauli_str = ["I"] * n
        pauli_str[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_str), coeffs=[coefs[i]])
    
    # Sanity: soma dos coefs
    print(f"Soma dos coefs mixer: {sum(coefs):.2f} (ideal ~{n} pra equilíbrio)")
    
    return h_mix

# Chame com heurística (default) ou uniforme=True; teste normalize_sum=True se quiser soma exata
hamiltoniano_m = hamiltoniano_de_mixer(valor_itens, peso_itens, uniforme=False, normalize_sum=False)
print("Hamiltoniano de mistura (corrigido e fixado):")
print(hamiltoniano_m)

Top 3 itens por v/w: idx [ 1 18 14], razoes [11.5, 5.357142857142857, 2.44]
Soma dos coefs mixer: 3.25 (ideal ~20 pra equilíbrio)
Hamiltoniano de mistura (corrigido e fixado):
SparsePauliOp(['IIIIIIIIIIIIIIIIIIII', 'XIIIIIIIIIIIIIIIIIII', 'IXIIIIIIIIIIIIIIIIII', 'IIXIIIIIIIIIIIIIIIII', 'IIIXIIIIIIIIIIIIIIII', 'IIIIXIIIIIIIIIIIIIII', 'IIIIIXIIIIIIIIIIIIII', 'IIIIIIXIIIIIIIIIIIII', 'IIIIIIIXIIIIIIIIIIII', 'IIIIIIIIXIIIIIIIIIII', 'IIIIIIIIIXIIIIIIIIII', 'IIIIIIIIIIXIIIIIIIII', 'IIIIIIIIIIIXIIIIIIII', 'IIIIIIIIIIIIXIIIIIII', 'IIIIIIIIIIIIIXIIIIII', 'IIIIIIIIIIIIIIXIIIII', 'IIIIIIIIIIIIIIIXIIII', 'IIIIIIIIIIIIIIIIXIII', 'IIIIIIIIIIIIIIIIIXII', 'IIIIIIIIIIIIIIIIIIXI', 'IIIIIIIIIIIIIIIIIIIX'],
              coeffs=[0.        +0.j, 0.0415879 +0.j, 1.        +0.j, 0.18200202+0.j,
 0.07543216+0.j, 0.0942029 +0.j, 0.0511509 +0.j, 0.07088847+0.j,
 0.03711559+0.j, 0.11594203+0.j, 0.10671937+0.j, 0.21195652+0.j,
 0.19323671+0.j, 0.11956522+0.j, 0.01571503+0.j, 0.21217391+0.j,
 0.01539855+0.j, 0.0931

In [6]:


# (Assumindo hamiltoniano_c, hamiltoniano_m do anterior)
def cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade, draw_output="mpl"):
    n_c = hamiltoniano_c.num_qubits
    n_m = hamiltoniano_m.num_qubits
    if n_c != n_m:
        raise ValueError(f"Mismatch qubits: custo={n_c}, mixer={n_m}. Deve ser igual!")
    if profundidade < 1:
        raise ValueError("Profundidade deve ser >=1 para QAOA.")
    
    n = n_c
    qc = QuantumCircuit(n)
    
    # Estado inicial: sobreposição uniforme
    qc.h(range(n))
    
    # Parâmetros (convenção: "gamma" em vez de "gama")
    gamma = ParameterVector("gamma", profundidade)
    beta = ParameterVector("beta", profundidade)
    
    # Camadas QAOA: alterna custo -> mixer
    for i in range(profundidade):
        evo_custo = PauliEvolutionGate(hamiltoniano_c, time=gamma[i])
        qc.append(evo_custo, range(n))
        evo_mixer = PauliEvolutionGate(hamiltoniano_m, time=beta[i])
        qc.append(evo_mixer, range(n))
    
    # Medição em Z-basis
    qc.measure_all()
    
    # Debug: métricas do circuito
    print(f"QAOA criado: n={n} qubits, p={profundidade} camadas, depth={qc.depth()}, size={qc.size()}")
    
    # Draw flexível
    try:
        if draw_output == "mpl":
            qc.draw("mpl")
        else:
            print(qc.draw("text"))  # Fallback textual
    except ImportError:
        print("Aviso: mpl não disponível; use draw_output='text'.")
        print(qc.draw("text"))
    
    return qc, gamma, beta  # Note: gamma agora

# Use p=2 pra debug rápido; 4 pra full
profundidade = 2  # Sugestão: comece baixo
qc, gamma, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade, draw_output="mpl")
print("Circuito QAOA criado com sucesso!")
# qc.draw("mpl")  # Já chamado na func

QAOA criado: n=20 qubits, p=2 camadas, depth=6, size=44
Circuito QAOA criado com sucesso!


In [7]:
# (Assumindo valor_itens, peso_itens, capacidade do topo)
def avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade, verbose=False):
    n = len(valor_itens)
    if not isinstance(bitstring, str) or len(bitstring) != n:
        if len(bitstring) < n:
            bitstring = bitstring.ljust(n, '0')  # Pad com 0 se curto
        else:
            raise ValueError(f"Bitstring '{bitstring}' deve ter len={n}; tem {len(bitstring)}")
    
    try:
        bits = np.array([int(b) for b in bitstring[::-1]])  # Vetorizado, little-endian fix
    except ValueError as e:
        raise ValueError(f"Bitstring inválida (não binária): {e}")
    
    valor_total = np.dot(bits, np.array(valor_itens))
    peso_total = np.dot(bits, np.array(peso_itens))
    valido = peso_total <= capacidade
    
    if verbose:
        print(f"Bitstring '{bitstring}' -> x={bits.tolist()[:5]}..., Valor={valor_total}, Peso={peso_total}, Válido={valido}")
    
    return valor_total, peso_total, valido

# Exemplo de teste (rode pra validar)
test_bitstring = '101000'[:6]  # Truncado pra demo; ajuste pra n=20
v_test, p_test = valor_itens[:6], peso_itens[:6]
cap_test = 100
print("Teste:")
avalia_bitstring('101000', v_test, p_test, cap_test, verbose=True)  # Ex: (44+72=116, 92+83=175 >100, False)

Teste:
Bitstring '101000' -> x=[0, 0, 0, 1, 0]..., Valor=112, Peso=151, Válido=False


(np.int64(112), np.int64(151), np.False_)

In [8]:


# (Assumindo qc, gamma, beta, qubo_matrix do anterior)
def custo_esperado(params, qc, gamma, beta, profundidade, backend, qubo_matrix, shots=2048, verbose=False):
    n = profundidade * 2
    if len(params) != n:
        raise ValueError(f"Params deve ter {n} elementos; tem {len(params)}")
    
    # Binding params (consistente com "gamma")
    params_dict = {gamma[i]: params[i] for i in range(profundidade)}
    params_dict.update({beta[i]: params[profundidade + i] for i in range(profundidade)})
    
    assigned_qc = qc.assign_parameters(params_dict)
    transpiled_qc = transpile(assigned_qc, backend, optimization_level=3)  # Otimiza gates
    
    try:
        job = backend.run(transpiled_qc, shots=shots)
        result = job.result()
        counts = result.get_counts()
    except Exception as e:
        print(f"Erro no backend run: {e}")
        return float('inf')  # Penaliza params ruins
    
    if not counts:
        if verbose:
            print("Aviso: Counts vazio; retornando inf")
        return float('inf')
    
    custo_total = 0.0
    total_shots = sum(counts.values())
    for bitstring, freq in counts.items():
        bits_list = [int(b) for b in bitstring[::-1]]
        if len(bits_list) != qubo_matrix.shape[0]:
            continue  
        bits = np.array(bits_list)
        custo = np.dot(bits, np.dot(qubo_matrix, bits))  # <- Full, vetorizado!
        custo_total += custo * (freq / total_shots)
    
    if verbose:
        print(f"<H_C> estimado: {custo_total:.2f} (shots={total_shots}, unique={len(counts)})")
    
    return custo_total

In [9]:
# Recrie o circuito com p=4 consistente (rode isso antes do teste!)
profundidade = 10  # Fix: Use o mesmo valor aqui e no teste
qc, gamma, beta = cria_qaoa(hamiltoniano_c, hamiltoniano_m, profundidade)

# Agora o teste corrigido
backend = BasicSimulator()
shots = 100  # Baixo pra debug rápido
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)  # 8 pra p=4

# Chame com args corretos (qubo_matrix incluso, como corrigido)
custo_test = custo_esperado(initial_params, qc, gamma, beta, profundidade, backend, qubo_matrix, shots=shots, verbose=True)
print(f"Custo teste: {custo_test:.2f}")

# Opcional: Rode com params zero pra baseline (uniform superposition)
zero_params = np.zeros(2 * profundidade)
custo_zero = custo_esperado(zero_params, qc, gamma, beta, profundidade, backend, qubo_matrix, shots=shots, verbose=True)
print(f"Custo baseline (γ=β=0, uniform): {custo_zero:.2f} (deve ser ~média QUBO, negativo médio)")

QAOA criado: n=20 qubits, p=10 camadas, depth=22, size=60
<H_C> estimado: -327483543.47 (shots=100, unique=100)
Custo teste: -327483543.47
<H_C> estimado: -335360038.32 (shots=100, unique=100)
Custo baseline (γ=β=0, uniform): -335360038.32 (deve ser ~média QUBO, negativo médio)


In [10]:
# (Assumindo qc, gamma, beta, qubo_matrix, backend, profundidade=4, valor_itens etc. do anterior)
shots = 2048  # Balance: preciso mas rápido; suba pra 10000 local
initial_params = np.random.uniform(0, np.pi, 2 * profundidade)

# Minimize com lambda pra passar qubo_matrix
result = minimize(
    fun=lambda p: custo_esperado(p, qc, gamma, beta, profundidade, backend, qubo_matrix, shots),
    x0=initial_params,
    method="COBYLA",
    options={"maxiter": 300, "tol": 1e-3, "disp": False}  # Mais iters, tol pra convergência
)

optimal_params = result.x
print("Parâmetros otimizados (γ, β):", optimal_params)
print("Custo mínimo <H_C>:", result.fun)  # Deve ser negativo ~ -1000 pra bom run

# Run final otimizado
params_dict = {gamma[i]: optimal_params[i] for i in range(profundidade)}
params_dict.update({beta[i]: optimal_params[profundidade + i] for i in range(profundidade)})
assigned_qc = qc.assign_parameters(params_dict)
transpiled_qc = transpile(assigned_qc, backend, optimization_level=3, coupling_map=None)
job = backend.run(transpiled_qc, shots=shots)
result_final = job.result()
counts = result_final.get_counts()

print("Contagens de bitstrings (otimizadas, top 5):")
print(dict(sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]))


top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 soluções (otimizadas):")
best_valor = -float('inf')
best_bitstring = None
num_validos = 0
total_freq_validos = 0
for bitstring, freq in top_results:
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    print(f"Bitstring: {bitstring}, Frequência: {freq}, Valor: {valor}, Peso: {peso}, Válido: {valido}")
    if valido:
        num_validos += 1
        total_freq_validos += freq


for bitstring, freq in counts.items():
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    if valido and valor > best_valor:
        best_valor = valor
        best_bitstring = bitstring
    if valido:
        num_validos += 1  
        total_freq_validos += freq

pct_validos = (total_freq_validos / shots) * 100 if shots > 0 else 0
print(f"% Amostras válidas: {pct_validos:.1f}% ({num_validos} unique válidas)")

if best_bitstring is None:
    print("Nenhuma solução válida encontrada. Aumente penalidade!")
else:
    print(f"\nMelhor solução válida: Bitstring {best_bitstring}, Valor {best_valor}")
    # Opcional: detalhes
    _, peso_best, _ = avalia_bitstring(best_bitstring, valor_itens, peso_itens, capacidade)
    print(f"Detalhes: Peso {peso_best} <= {capacidade}")

Parâmetros otimizados (γ, β): [2.98559372 1.0122483  2.88425159 1.24625951 3.45270935 0.80775337
 0.696977   1.74370861 0.84759134 2.38635316 2.64064472 0.25799564
 0.65244642 2.08178305 0.59228413 0.96567668 1.92507793 1.00209482
 2.98231557 1.89034247]
Custo mínimo <H_C>: -330045257.6274414
Contagens de bitstrings (otimizadas, top 5):
{'01000101111110001001': 2, '01000011101011100000': 2, '01011000010111100010': 2, '01100110110010011000': 1, '10101101101110010110': 1}
Top 10 soluções (otimizadas):
Bitstring: 01000101111110001001, Frequência: 2, Valor: 544, Peso: 452, Válido: True
Bitstring: 01000011101011100000, Frequência: 2, Valor: 411, Peso: 457, Válido: True
Bitstring: 01011000010111100010, Frequência: 2, Valor: 449, Peso: 464, Válido: True
Bitstring: 01100110110010011000, Frequência: 1, Valor: 496, Peso: 469, Válido: True
Bitstring: 10101101101110010110, Frequência: 1, Valor: 611, Peso: 564, Válido: True
Bitstring: 10111000011001001000, Frequência: 1, Valor: 463, Peso: 523, Váli

In [20]:
def knapsack_dp(values, weights, capacity):
    n = len(values)
    #taabela DP: dp[i][w] 
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        for w in range(capacity + 1):
            if weights[i-1] <= w:
                # Max 
                dp[i][w] = max(dp[i-1][w], dp[i-1][w - weights[i-1]] + values[i-1])
            else:
                dp[i][w] = dp[i-1][w]
    
    max_value = dp[n][capacity]
    
    # Backtrack
    bitstring = ['0'] * n
    w = capacity
    for i in range(n, 0, -1):
        if weights[i-1] <= w and dp[i][w] == dp[i-1][w - weights[i-1]] + values[i-1]:
            bitstring[i-1] = '1' #inclui o item
            w -= weights[i-1] #atualiza capacidade
    
    peso_total = sum(weights[i] for i in range(n) if bitstring[i] == '1')
    return max_value, ''.join(bitstring), peso_total

max_val, bitstr_otima, peso_otimo = knapsack_dp(valor_itens, peso_itens, capacidade)
print(f"Ótimo exato (DP): {max_val}")
print(f"Bitstring ótima: {bitstr_otima}")
print(f"Peso total ótimo: {peso_otimo} (<= {capacidade})")

Ótimo exato (DP): 1024
Bitstring ótima: 11111111111110101011
Peso total ótimo: 871 (<= 878)


In [27]:
print("Melhor valor encontrado usando QAOA e bitstring:")
print(f"{best_valor} e {best_bitstring}\n")

print("Melhor valor encontrado usando DP")
print(f"{max_val}\n")

print("ratio Ratio QAOA vs DP:")
aprox = (best_valor / max_val) * 100
print(f"Ratio: {aprox:.2f}%")

print("Gap entre DP e QAOA(Hibrido)") 
gap = ((max_val - best_valor) / max_val) * 100
print(f"Gap: {gap:.2f}%")


Melhor valor encontrado usando QAOA e bitstring:
956 e 11000111111011111111

Melhor valor encontrado usando DP
1024

ratio Ratio QAOA vs DP:
Ratio: 93.36%
Gap entre DP e QAOA(Hibrido)
Gap: 6.64%
